In [1]:
# Import required modules
import pandas as pd
from selenium import webdriver
import time

In [2]:
# Read data to get adobestock image link
df = pd.read_csv("_Bucket 3.1 - Sheet1.csv")
adobestock_df = df[df["Image Name"].str.contains("adobestock", case=False, na=False)]

# Create image page link
adobestock_df["created_link"] = "https://stock.adobe.com/" + adobestock_df["Image Name"].str.split("_").str[-1].str.replace(".jpeg", "")

<ipython-input-2-e95e34a5100e>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  adobestock_df["image_page_link"] = "https://stock.adobe.com/" + adobestock_df["Image Name"].str.split("_").str[-1].str.replace(".jpeg", "")
<ipython-input-2-e95e34a5100e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adobestock_df["image_page_link"] = "https://stock.adobe.com/" + adobestock_df["Image Name"].str.split("_").str[-1].str.replace(".jpeg", "")


In [3]:
# Initialize the drivers
driver = webdriver.Chrome("/home/faysal/Documents/utilities/chromedriver")

# Empty list of variables to scrape
title = []
src_link = []
category = []
keywords = []
created_link = []

for url in adobestock_df.image_page_link.unique():
    driver.get(url)
    time.sleep(3)
    
    # Get image page link
    created_link.append(url)
    
    # Scrape title
    try:
        try:
            # Is see more button exists
            see_more_button = driver.find_element_by_css_selector(".blue.bleu-de-france-text.js-trimmer-toggle.text-sregular")
            see_more_button.click()
            title.append(driver.find_element_by_css_selector(".no-margin.beta.js-details-title").text.replace("See Less", "").strip())
        except:
            title.append(driver.find_element_by_css_selector(".no-margin.beta.js-details-title").text.strip())
    except:
        title.append("na")
    
    # Scrape image source link
    try:
        src_link.append(driver.find_element_by_css_selector("#details-enlarged-image").get_attribute("src"))
    except:
        src_link.append("na")
    
    # Scrape keywords
    try:
        try:
            view_all_button = driver.find_element_by_css_selector(".js-keywords-see-all.badge.badge--default.badge--large.grey.nurse")
            view_all_button.click()
            keywords.append(driver.find_element_by_css_selector(".details-keywords-list-original").text.replace("\n", ", ").strip())
        except:
            keywords.append(driver.find_element_by_css_selector(".details-keywords-list-original").text.replace("\n", ", ").strip())
    except:
        keywords.append("na")
    
    
    # Scrape category
    try:
        category.append(driver.find_element_by_css_selector(".js-category-link").text.strip())
    except:
        category.append("na")
    
    time.sleep(2)
    

    
# Create a temp dataframe
temp_df = pd.DataFrame({
    "image_page_link":image_page_link,
    "title":description,
    "category":category,
    "keywords":keywords,
    "source_link":src_link
})

temp_df.to_csv("bucket_3.1_adobestock_scraped.csv", index=None)